- [o] 공통 데이터 전처리 완료
- [o] 개별 모델 실험을 통한 피처 엔지니어링 진행 완료 `10/15: columns.json → use_yn 체크후 next 진행`
- [-] 최종 피처 데이터를 통한 모델 성능 비교 및 최적화 
- [-] 최종 모델 선택

In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path("").resolve().parent.parent))

In [ ]:
import json
from preprocess import preprocessing_data, featuring_data, drop_feature

ROOT_DIR = Path("").resolve().parent.parent
DATA_DIR = ROOT_DIR / "data"
DATA_PATH = DATA_DIR / "analy_data.csv"
SAVE_FILE = DATA_DIR / "prep_data_v1.csv"

def rename_to_kor(df):
    """columns.json의 name으로 한글 컬럼명 매핑"""
    rename_map = {
        col: meta["name"]
        for cat in columns_dict.values()
        for col, meta in cat.items()
        if col in df.columns
    }
    return df.rename(columns=rename_map)

with open("modules/columns.json", "r", encoding="utf-8") as f:
    columns_dict = json.load(f)

df_merge = preprocessing_data(DATA_PATH)
df_merge = featuring_data(df_merge)
df_merge = drop_feature(df_merge)

# 최종 피처 선택
fnl_col = df_merge.columns.tolist()
df = df_merge[fnl_col]

# 한글 매핑
df_merge_kor = rename_to_kor(df)
df = df_merge_kor.copy()

(전처리)데이터 크기: (89822, 73) (2개 제외)


,exmprs_no,age,sex,CTPRVN_CODE,SPOT_NO,wt_p,mbhld_co,reside_adult_co,fma_04z1,fma_13z1,...,smc_09z2,smc_10z2,sob_01z1,sob_02z1,soa_01z1,soa_06z2,soa_07z1,sod_02z3,age_group,is_single
0,2024111170001700102002201,31.0,1,11,00102,380.47075,1.0,1.0,3.0,12000.0,...,NaN,2.0,7.0,1.0,1.0,3.0,2.0,5.0,30.0,1
1,2024111170001700102005901,37.0,1,11,00102,380.47075,3.0,2.0,3.0,NaN,...,NaN,2.0,5.0,1.0,1.0,4.0,2.0,1.0,30.0,0
2,2024111170001700102005902,36.0,2,11,00102,376.40383,3.0,2.0,3.0,NaN,...,NaN,2.0,6.0,1.0,1.0,4.0,2.0,1.0,30.0,0
3,2024111170001700102017001,36.0,1,11,00102,380.47075,2.0,2.0,3.0,8000.0,...,NaN,2.0,7.0,1.0,1.0,3.0,2.0,1.0,30.0,0
4,2024111170001700191001801,64.0,1,11,00191,119.45059,1.0,1.0,1.0,NaN,...,NaN,3.0,3.0,3.0,2.0,NaN,NaN,5.0,60.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89817,2024503970006201711015701,49.0,1,50,01711,51.53645,1.0,1.0,3.0,5000.0,...,NaN,2.0,7.0,3.0,1.0,2.0,1.0,2.0,40.0,1
89818,2024503970006201711018701,77.0,1,50,01711,30.93451,5.0,5.0,3.0,7000.0,...,NaN,3.0,5.0,1.0,2.0,NaN,NaN,1.0,70.0,0
89819,2024503970006201711021602,64.0,1,50,01711,30.93451,3.0,3.0,3.0,3000.0,...,NaN,2.0,5.0,1.0,1.0,9.0,2.0,1.0,60.0,0
89820,2024503970006201711021603,40.0,1,50,01711,51.53645,3.0,3.0,3.0,3000.0,...,NaN,3.0,6.0,3.0,2.0,NaN,NaN,5.0,40.0,0


In [ ]:
# 학습/검증/테스트!!!!!!! 데이터 분리
if 'churn' in df.columns: TARGET_NM = 'churn'
else: TARGET_NM = '이탈여부'

from sklearn.model_selection import train_test_split

# 데이터 분리
X = df.drop(TARGET_NM, axis=1)
X[X.select_dtypes(exclude=['number']).columns] = X.select_dtypes(exclude=['number']).astype(float)
y = df[TARGET_NM]

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=2)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, '\n')

# 필요시 인코딩 (CatBoost)

In [ ]:
# 모델 정의
catb_params = {}
xgb_params = {}
lgb_params = {}
rf_params = {}
hgb_params = {}

models = {
    "DecisionTree": Catboost(random_state=2, **catb_params),
    "ExtraTrees": XGBClassifier(random_state=2, **xgb_params),
    "ExtraTrees": LGBClassifier(random_state=2, **lgb_params),
    "RandomForest": RandomForestClassifier(random_state=2, **rf_params),
    "HistGradientBoosting": HistGradientBoostingClassifier(random_state=2, **hgb_params)
}

# 학습 및 평가
import time 
results = []
for name, model in models.items():
    
    start = time.perf_counter()
    model.fit(X_train, y_train)
    train_time_sec = time.perf_counter() - start
    print(f"{name} | 학습시간(초): {train_time_sec:.3f}")

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:,1]
    
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    pr_auc = average_precision_score(y_test, y_prob)
    results.append((name, acc, auc, pr_auc))

# 결과 저장
df_results = pd.DataFrame(results, columns=['Model', 'Accuracy', 'ROC_AUC', 'PR_AUC']).sort_values('ROC_AUC', ascending=False)
print('\n')
print(df_results)

In [ ]:
# 최종 모델 선택해서 gridsearch 진행

In [ ]:
# GridSearchCV, cross_val_score
from sklearn.model_selection import ParameterGrid, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score
import numpy as np
import time

# 베스트 모델  
best_model = df_best[df_best['BEST_ROC_AUC']==df_best['BEST_ROC_AUC'].max()]['모델'].item()

# 파라미터 후보
param_grid = {}
best_params = df_best[df_best['BEST_ROC_AUC']==df_best['BEST_ROC_AUC'].max()]['최적파라미터'].item()
for param in best_params.keys():
    param_1 = int(best_params[param]*0.7) if param not in ('learning_rate', 'l2_regularization') else best_params[param]*0.7
    param_2 = best_params[param]
    param_3 = int(best_params[param]*1.5) if param not in ('learning_rate', 'l2_regularization') else best_params[param]*1.5
    param_grid[param] = [param_1, param_2]
    
# 파라미터 조합 생성
param_list = list(ParameterGrid(param_grid))
print(f"총 {len(param_list)}개 조합 탐색\n")

best_score = -1
best_params_ = None
best_model_ = None

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)

for i, params in enumerate(param_list, 1):
    
    print(f"[{i}/{len(param_list)}] 현재 파라미터: {params}")
    start_time = time.time()
    
    # 최적 모델
    model = models[best_model].set_params(**params)
    acc_list, auc_list, pr_auc_list = [], [], []
    
    # 교차검증 수행 (5-Fold)
    for fold, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train), 1):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_val)
        y_prob = model.predict_proba(X_val)[:, 1]

        acc = accuracy_score(y_val, y_pred)
        auc = roc_auc_score(y_val, y_prob)
        pr_auc = average_precision_score(y_val, y_prob)

        acc_list.append(acc)
        auc_list.append(auc)
        pr_auc_list.append(pr_auc)

        print(f"Fold {fold}: acc={acc:.4f}, auc={auc:.4f}, pr_auc={pr_auc:.4f}")
    
    mean_acc = np.mean(acc_list)
    mean_auc = np.mean(auc_list)
    mean_pr_auc = np.mean(pr_auc_list)
    elapsed = time.time() - start_time

    print(f"평균 acc={mean_acc:.4f}, auc={mean_auc:.4f}, pr_auc={mean_pr_auc:.4f} | 소요시간: {elapsed:.1f}초\n")

    # 최고 모델 업데이트(AUC 기준)
    if mean_auc > best_score:
        best_score = mean_auc
        best_params_ = params
        best_model_ = model

print("탐색 완료")
print("Best Params:", best_params_)
print("Best Test Accuracy:", best_score)

In [ ]:
# 최적 모델 재학습 및 평가
best_model_.fit(X_train, y_train)

# test
y_pred = best_model_.predict(X_test)
y_prob = best_model_.predict_proba(X_test)[:, 1]

test_acc = accuracy_score(y_test, y_pred)
test_auc_roc = roc_auc_score(y_test, y_prob)
test_pr_auc = average_precision_score(y_test, y_prob)

# train
train_y_pred = best_model_.predict(X_train)
train_y_prob = best_model_.predict_proba(X_train)[:, 1]

train_acc = accuracy_score(y_train, train_y_pred)
train_auc_roc = roc_auc_score(y_train, train_y_prob)
train_pr_auc = average_precision_score(y_train, train_y_prob)

print(f"\n학습 성능: train_acc={train_acc:.4f}, train_auc_roc={train_auc_roc:.4f}, train_pr_auc={train_pr_auc:.4f}")
print(f"\n평가 성능: test_acc={test_acc:.4f}, test_auc_roc={test_auc_roc:.4f}, test_pr_auc={test_pr_auc:.4f}") # 테스트 셋으로

In [ ]:
from sklearn.inspection import permutation_importance
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
font_path = r'C:\Windows\Fonts\HMFMMUEX.TTC'
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)
plt.rcParams['axes.unicode_minus'] = False

# 예측 및 평가
y_pred = best_model_.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

# 변수중요도
result = permutation_importance(best_model_, X_test, y_test, n_repeats=10, random_state=2, n_jobs=-1)

# 중요도 정렬
sorted_idx = result.importances_mean.argsort()[::-1]

plt.figure(figsize=(8, 5))
plt.barh(np.array(X_test.columns)[sorted_idx][:15], result.importances_mean[sorted_idx][:15])
plt.xlabel("Permutation Importance")
plt.ylabel("Feature")
plt.title("Top 15 Feature Importances (Permutation-based)")
plt.gca().invert_yaxis()
plt.show()

# 혼동행렬
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# ROC-AUC
y_prob = best_model_.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_prob)
fpr, tpr, _ = roc_curve(y_test, y_prob)

# PR-AUC
pr_auc = average_precision_score(y_test, y_prob)
prec, recall, _ = precision_recall_curve(y_test, y_prob)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].plot(fpr, tpr, label=f"ROC-AUC = {roc_auc:.3f}")
axes[0].plot([0,1],[0,1],'--')
axes[0].set_xlabel("FPR")
axes[0].set_ylabel("TPR")
axes[0].set_title("ROC Curve")
axes[0].legend()

axes[1].plot(recall, prec, label=f"PR-AUC = {pr_auc:.3f}")
axes[1].set_xlabel("Recall")
axes[1].set_ylabel("Precision")
axes[1].set_title("Precision-Recall Curve")
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# 학습 데이터셋 + y_pred -> 예측 결과 데이터셋 저장
# 학습 결과 pkl 저장